In [1]:
from langchain_chroma import Chroma

### 간단한 예시

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()  # API 키 입력
os.environ["LANGCHAIN_API_KEY"] = "..."  # API 키 입력
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

In [3]:
from langchain_aws import ChatBedrock  # Bedrock 모델 사용

# Bedrock 모델 설정
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0)
)

# def 추출(가지고 있는 정보, 알고 싶은 내용): -> 리트리버(Retreiver)
    # 얼마나_비슷한지_판단(가지고 있는 정보, 알고 싶은 내용) -> 백터 스토어(Vector Store)


# 필요한 정보 = 추출(가지고 있는 정보, 알고 싶은 내용)

# result = model.invoke(필요한 정보 + "다음 미국 대선 후보는 누구인가요?")
# print(result)


content='2024년 미국 대선의 주요 후보들은 아직 공식적으로 확정되지 않았습니다. 하지만 현재 상황을 보면:\n\n1. 민주당:\n   - 조 바이든 현 대통령: 재선 출마를 선언했습니다.\n\n2. 공화당:\n   - 도널드 트럼프 전 대통령: 출마를 선언했습니다.\n   - 론 디샌티스 플로리다 주지사: 출마할 가능성이 높습니다.\n   - 니키 헤일리 전 유엔 대사: 출마를 선언했습니다.\n\n그 외에도 여러 후보들이 출마를 고려하거나 선언할 수 있습니다. 대선이 가까워질수록 더 명확한 후보 구도가 형성될 것입니다. 정확한 후보 명단은 각 당의 예비선거를 통해 결정될 것입니다.' additional_kwargs={'usage': {'prompt_tokens': 27, 'completion_tokens': 310, 'total_tokens': 337}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} response_metadata={'usage': {'prompt_tokens': 27, 'completion_tokens': 310, 'total_tokens': 337}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'} id='run-bfec0ee4-2fb3-457b-81c6-d0d3996aa1f3-0' usage_metadata={'input_tokens': 27, 'output_tokens': 310, 'total_tokens': 337}


위의 상황을 보면 최신 정보는 담기지 않아 곤혹을 겪을 수 있다.    
이럴 때 사용할 수 있는 방법이 벡터 스토어(단어와 단어 사이의 관계를 이해할 수 있게 해줌), 리트리버(저장된 데이터를 불러오는 기능)들이 필요하다.

---

### 문서 생성

기존의 체인   
입력 | 모델 | 출력   

RAG 사용   
**정보** + 입력 | 모델 | 출력   

유사한_문장(VectorStore) = 유사도_측정기(정보들, 내가_궁금한_문장) # Vector Store    
유효한_정보 = 정보 추출기(정보, 내가_궁금한_문장) -> Retreiver(docs, question)

In [4]:
from langchain_core.documents import Document

# 개 고양이에 대한 간단한 정보 넣어줌
documents = [
    Document(
        page_content="개는 충성스럽고 친절한 동반자로 알려져 있습니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="토끼는 사회적인 동물로, 뛰어놀 공간이 많이 필요합니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
]


---

### 환경 변수 설정

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()  # API 키 입력
os.environ["LANGCHAIN_API_KEY"] = "..."  # API 키 입력
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

---

### 백터 스토어 설정

In [5]:
from langchain_chroma import Chroma
from langchain_aws import BedrockEmbeddings

# Amazon Titan 임베딩 모델 사용
embedding_model = BedrockEmbeddings(
    model_id='amazon.titan-embed-text-v1',
)

# 벡터 스토어 생성
vectorstore = Chroma.from_documents(
    documents,
    embedding=embedding_model,
)


강아지란 단어가 어떤 수치의 조합으로 이뤄졌는지 알아보자

In [7]:
output = embedding_model.embed_query("강아지")
output

[-0.014892578125,
 0.04052734375,
 -0.1328125,
 -0.2275390625,
 0.00830078125,
 -0.29296875,
 0.31640625,
 -1.811981201171875e-05,
 -0.27734375,
 -0.57421875,
 0.1337890625,
 0.5390625,
 0.0732421875,
 -0.3515625,
 0.419921875,
 0.064453125,
 0.3203125,
 -0.7578125,
 -0.7265625,
 0.08349609375,
 0.1767578125,
 -0.34375,
 -0.0439453125,
 0.72265625,
 -0.515625,
 0.8984375,
 0.36328125,
 0.09375,
 0.37109375,
 -0.65234375,
 -0.2490234375,
 1.0390625,
 -0.34765625,
 -1.03125,
 0.671875,
 -0.28125,
 0.04248046875,
 -1.3046875,
 -0.232421875,
 -0.10986328125,
 -0.22265625,
 0.255859375,
 1.0234375,
 0.5234375,
 0.41796875,
 0.162109375,
 0.08154296875,
 0.32421875,
 0.46875,
 0.271484375,
 0.02880859375,
 0.380859375,
 -1.1796875,
 -1.0703125,
 0.16015625,
 -0.1455078125,
 -0.1962890625,
 -0.1318359375,
 -1.6328125,
 -0.068359375,
 0.16015625,
 0.0010986328125,
 0.80078125,
 0.43359375,
 0.59375,
 -0.408203125,
 -0.043212890625,
 -0.5390625,
 -0.265625,
 -0.2109375,
 0.09912109375,
 0.20507

1536개의 수치로 이뤄져있는 것을 알 수 있다.

In [8]:
len(output)

1536

In [11]:
output = embedding_model.embed_query("개는 충성스럽고 친절한 동반자로 알려져 있습니다.")
output

[0.48046875,
 -0.3671875,
 -0.212890625,
 0.5078125,
 0.4140625,
 -0.1875,
 0.330078125,
 -0.0002536773681640625,
 -0.13671875,
 -0.6484375,
 0.2138671875,
 0.578125,
 -0.1064453125,
 0.2294921875,
 0.85546875,
 -0.1494140625,
 0.435546875,
 0.001708984375,
 -0.6015625,
 0.7890625,
 0.232421875,
 -0.9140625,
 -0.53515625,
 1.3125,
 0.10595703125,
 0.2275390625,
 0.26171875,
 -0.96875,
 -0.263671875,
 -0.8359375,
 -0.244140625,
 1.375,
 0.60546875,
 -1.2890625,
 0.236328125,
 0.8046875,
 -0.1533203125,
 -0.609375,
 0.12353515625,
 -0.515625,
 -0.169921875,
 0.2431640625,
 1.0703125,
 0.208984375,
 0.765625,
 0.80859375,
 0.061279296875,
 0.80078125,
 0.032470703125,
 -0.49609375,
 0.44140625,
 0.640625,
 0.005096435546875,
 0.08740234375,
 -0.236328125,
 0.388671875,
 -0.328125,
 -0.07421875,
 0.126953125,
 -0.040771484375,
 -0.0498046875,
 -0.392578125,
 0.26953125,
 0.84765625,
 0.439453125,
 0.1650390625,
 -0.0020751953125,
 -0.46484375,
 -0.01318359375,
 -0.609375,
 -0.59375,
 -0.54

이처럼 벡터의 개수는 고정적인 것을 알 수 있다.

In [12]:
len(output)

1536

아래를 보면 어떤 문장과 가장 유사한지 확인할 수 있다.

In [28]:
result = vectorstore.similarity_search("meow")
result

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='개는 충성스럽고 친절한 동반자로 알려져 있습니다.')]

In [29]:
# "고양이"와 유사한 문서 검색
result = vectorstore.similarity_search("고양이")
for doc in result:
    print(doc.page_content)

고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.
금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다.
개는 충성스럽고 친절한 동반자로 알려져 있습니다.
앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.


값이 작을 수록 유사도가 높다고 볼 수 있다.

In [30]:
# 유사도 점수와 함께 검색
result_with_score = vectorstore.similarity_search_with_score("고양이")
for doc, score in result_with_score:
    print(f"문서: {doc.page_content}, 유사도 점수: {score}")

문서: 고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다., 유사도 점수: 423.8027038574219
문서: 금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다., 유사도 점수: 518.2442626953125
문서: 개는 충성스럽고 친절한 동반자로 알려져 있습니다., 유사도 점수: 533.6537475585938
문서: 앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다., 유사도 점수: 575.236083984375


In [32]:
# 쿼리를 벡터로 변환하여 검색
embedding = embedding_model.embed_query("고양이")
embedding

[1.1171875,
 0.3671875,
 0.1826171875,
 0.33984375,
 -0.177734375,
 -0.345703125,
 0.162109375,
 -0.0010528564453125,
 -0.31640625,
 -1.140625,
 -0.138671875,
 0.45703125,
 -0.26171875,
 0.11962890625,
 0.703125,
 0.0301513671875,
 0.0380859375,
 -1.671875,
 -1.1796875,
 0.39453125,
 -0.19140625,
 -0.39453125,
 -1.3359375,
 1.609375,
 0.25,
 0.67578125,
 0.7734375,
 -1.4765625,
 -0.578125,
 -1.265625,
 0.80078125,
 0.08935546875,
 -0.67578125,
 -1.9921875,
 -0.2099609375,
 -0.412109375,
 0.1513671875,
 -1.140625,
 -0.07080078125,
 0.486328125,
 -0.6015625,
 0.205078125,
 1.390625,
 -0.291015625,
 0.453125,
 -0.1435546875,
 0.28515625,
 0.458984375,
 0.0478515625,
 -0.384765625,
 0.259765625,
 1.2890625,
 -1.09375,
 -0.83984375,
 0.041748046875,
 0.53515625,
 -1.328125,
 -0.171875,
 -1.40625,
 0.024658203125,
 0.3515625,
 0.71484375,
 0.359375,
 0.796875,
 0.4296875,
 -0.5625,
 0.00102996826171875,
 -0.10693359375,
 -0.6640625,
 -0.3359375,
 -0.26953125,
 -0.259765625,
 1.0703125,
 0.13

In [33]:
# 쿼리를 벡터로 변환하여 검색
embedding = embedding_model.embed_query("고양이")
vector_result = vectorstore.similarity_search_by_vector(embedding)
for doc in vector_result:
    print(doc.page_content)


고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.
금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다.
개는 충성스럽고 친절한 동반자로 알려져 있습니다.
앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.


In [34]:
vectorstore.similarity_search_by_vector(embedding)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='금붕어는 초보자에게 인기가 많은 반려동물로, 관리가 비교적 간단합니다.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='개는 충성스럽고 친절한 동반자로 알려져 있습니다.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.')]

In [35]:
from langchain_core.runnables import RunnableLambda

# 유사도 검색을 통해 상위 1개(k=1)의 결과를 반환하는 리트리버 생성
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

# 리트리버로 "고양이", "상어" 쿼리 처리
results = retriever.batch(["고양이", "상어"])
for result in results:
    for doc in result:
        print(doc.page_content)
# 각각 고양이, 상어와 연관성이 가장 높은 것을 찾아주게 된다.


고양이는 독립적인 반려동물로 자신의 공간을 즐기는 경향이 있습니다.
앵무새는 인간의 말을 흉내낼 수 있는 지능적인 새입니다.


---

이제 리트리버를 사용해 체인으로 엮어준다.

### RAG 체인 생성

In [36]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Amazon Bedrock LLM 설정 (Claude 모델 사용)
llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
)

# 프롬프트 템플릿 정의
message = """
질문에 주어진 문맥만을 사용하여 답변하세요.

질문:
{question}

문맥:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

# RAG 체인 설정: 리트리버로 문맥을 가져와 질문과 함께 LLM에 전달
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

# 질문에 대한 답변 생성
response = rag_chain.invoke("고양이에 대해 알려주세요")
print(response.content)


주어진 문맥에 따르면, 고양이는 독립적인 반려동물입니다. 그들은 자신만의 공간을 즐기는 성향이 있습니다. 이 정보는 고양이의 특성 중 하나를 설명하고 있습니다.
